In [1]:

import os
os.environ["OPENCV_VIDEOIO_MSMF_ENABLE_HW_TRANSFORMS"] = "0"
import cv2
import numpy as np
import glob
import time
from IPython.display import clear_output
import json

### https://markhedleyjones.com/projects/calibration-checkerboard-collection
Used a 8 x 6 Checker board with 25mm titles an A4 paper in full pages without margin

In [2]:
# cap = cv2.VideoCapture(0, cv2.CAP_MSMF) # u can use cv2.CAP_DSHOW / cv2.CAP_MSMF / cv2.CAP_VFW / cv2.CAP_ANY

cap = cv2.VideoCapture(1,cv2.CAP_MSMF)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 3840)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 2160)
#1080p
# q
if not cap.isOpened():
    raise Exception("Could not open video device")

while True:
    #capture an image and show it 
    ret, frame = cap.read()
    if not ret:
        raise Exception("Could not read frame")
    #show the image
    resized = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    cv2.imshow("resized", resized)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()


In [3]:
ret, frame = cap.read()
save_path = 'calibration_images'
if not os.path.exists(save_path):
    os.mkdir(save_path)
img_counter = 20
#checkerboars settings
CHECKERBOARD = (4,4)
do_wait = False
while True:
    ret, frame = cap.read()
    if ret:
        gray = cv2.cvtColor(frame ,cv2.COLOR_BGR2GRAY)
        result, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
        if result == True:
            cv2.imwrite(os.path.join(save_path, 'calibration_image_{}.png'.format(img_counter)), frame)
            img_counter += 1
            clear_output()
            #write the corners on the image
            cv2.drawChessboardCorners(frame, CHECKERBOARD, corners, result)
        resized = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        cv2.imshow("resized", resized)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
        #if the user presses 'c' then continue
        if cv2.waitKey(1) & 0xFF == ord('c'):
            continue
    #refocus the camera
    else:
        break

In [4]:
img_path = 'calibration_images_close_up'
imgs_path = glob.glob(img_path + '/*.png')
for img_path in imgs_path:
    print(img_path)

calibration_images_close_up\calibration_image_51.png
calibration_images_close_up\calibration_image_52.png
calibration_images_close_up\calibration_image_53.png
calibration_images_close_up\calibration_image_54.png
calibration_images_close_up\calibration_image_55.png
calibration_images_close_up\calibration_image_56.png
calibration_images_close_up\calibration_image_57.png
calibration_images_close_up\calibration_image_58.png
calibration_images_close_up\calibration_image_59.png
calibration_images_close_up\calibration_image_60.png
calibration_images_close_up\calibration_image_61.png
calibration_images_close_up\calibration_image_62.png
calibration_images_close_up\calibration_image_63.png
calibration_images_close_up\calibration_image_64.png
calibration_images_close_up\calibration_image_65.png
calibration_images_close_up\calibration_image_66.png
calibration_images_close_up\calibration_image_67.png
calibration_images_close_up\calibration_image_68.png
calibration_images_close_up\calibration_image_

In [5]:
# Defining the dimensions of checkerboard
CHECKERBOARD = (4,4)
CHECKERBOARD_CELL_SIZE = 0.005 # in meters
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 
 
 
# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

for fname in imgs_path:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
     
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker boardq
    """
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
         
        imgpoints.append(corners2)
 
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
        # print 'press any key to continue'
        img = cv2.putText(img, 'press any key to continue', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('img',img)
    cv2.waitKey(0)
 
cv2.destroyAllWindows()
 
h,w = img.shape[:2]
 
"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
 
print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs : \n")
print(rvecs)
print("tvecs : \n")
print(tvecs)

Camera matrix : 

[[3.19676036e+03 0.00000000e+00 1.99584016e+03]
 [0.00000000e+00 3.23444976e+03 1.16263597e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist : 

[[ 1.63529553e-01 -5.30003464e-01 -2.21510102e-03 -5.04017208e-04
   1.39959228e-01]]
rvecs : 

(array([[-0.40530858],
       [-0.51894896],
       [ 0.4680111 ]]), array([[-0.4283058 ],
       [-0.51001034],
       [ 0.46058384]]), array([[-0.49700305],
       [-0.13179695],
       [ 0.4426493 ]]), array([[-0.51009732],
       [-0.14258097],
       [ 0.44272302]]), array([[-0.5043102 ],
       [-0.14807579],
       [ 0.44856454]]), array([[-0.50214696],
       [-0.14786718],
       [ 0.44992602]]), array([[-0.50202346],
       [-0.14853575],
       [ 0.44934921]]), array([[-0.50198498],
       [-0.15478306],
       [ 0.44721587]]), array([[-0.49853291],
       [-0.154638  ],
       [ 0.44668397]]), array([[-0.21006057],
       [-0.52862771],
       [-1.14018386]]), array([[ 0.04763346],
       [ 0.01507334],
       

In [6]:
#save the camera matrix and distortion coefficients
with open('calibration_data_close_up.json', 'w') as outfile:
    json.dump({'mtx': mtx.tolist(), 'dist': dist.tolist()}, outfile)
#reconstruct the camera matrix and distortion coefficients
with open('calibration_data_close_up.json', 'r') as infile:
    reconstruction = json.load(infile)
    new_mtx = np.array(reconstruction['mtx'])
    new_dist = np.array(reconstruction['dist'])
cv2.destroyAllWindows()
print('done')

done
